# How to Deploy an App

Before building your app for real devices, you'll need your backend server accessible over the internet. For quick testing and development, **ngrok** provides an easy way to tunnel your local FastAPI server with a public URL.

## Step 1: Using ngrok

---

### Benefits of Using ngrok

* Test your full app workflow (frontend + backend) on real devices
* No need to deploy to cloud hosting during development
* Simple, secure, and free for basic use

**Note:** ngrok URLs reset each time you restart the tunnel. Be sure to update your API URL if needed.

---

### Install ngrok

**On macOS (with Homebrew):**

```bash
brew install ngrok
```

**On Windows (with Chocolatey):**

```bash
choco install ngrok
```

Alternatively, download directly from [ngrok.com](https://ngrok.com/download).

## Step 2: Create a Free ngrok Account

While ngrok works without an account, creating a free account unlocks extra features like:

- Longer session times
- Custom subdomains
- Better tunnel stability

---

**Sign Up for Free**

Visit [ngrok.com/signup](https://dashboard.ngrok.com/signup) to create your account.

After signing up, you'll receive a personal **authtoken**, which links ngrok to your account.

---

**Save Your ngrok Authtoken**

In your terminal, run:

```bash
ngrok config add-authtoken YOUR_AUTHTOKEN_HERE
```

Replace `YOUR_AUTHTOKEN_HERE` with the token from your ngrok dashboard.

This saves your credentials, so you don’t have to enter the token every time you start a tunnel.

---

Once saved, you can use ngrok normally with full access to account features.


## Step 3: Start Your FastAPI Server Locally

Before tunneling with ngrok, your FastAPI backend needs to be running.

---

**Run Your Backend**

Make sure you're inside your backend project folder, then activate your virtual environment and run:

```bash
uvicorn app.main:app --reload
```

---

**Confirm It's Running**

You should see something like:

```
Uvicorn running on http://127.0.0.1:8000
```

If that's showing, your backend is ready and listening on `localhost` port `8000`.

Next, we'll expose this server to the internet using ngrok.


## Step 4: Start ngrok to Tunnel Your Backend

In a **separate terminal window**, navigate to your backend project folder (if you're not already there), then run:

```bash
ngrok http 8000
```

---

**What You Should See**

ngrok will output something like:

```
Forwarding                    https://a1b2c3d4.ngrok.io -> http://localhost:8000
```

Copy that **https URL** — this is now your public-facing backend endpoint. You’ll use this in your app to send API requests.


## Step 5: Update Your React Native App to Use the ngrok URL

In your React Native project, open your `ConversionScreen.js` file.

Find this line:

```js
const API_URL = "YOU_WILL_CHANGE_THIS";
```

Replace `"YOU_WILL_CHANGE_THIS"` with your ngrok **https URL** from earlier. For example:

```js
const API_URL = "https://a1b2c3d4.ngrok.io";
```

---

**That’s It!**

Your mobile app can now send requests to your FastAPI backend — even though it's running locally — thanks to ngrok’s public tunnel.


## Step 6: Using Expo Go to Run the Frontend

Make sure your Android emulator is running.

In a new terminal window, navigate to your frontend project folder and start Expo:

```bash
npx expo start
```

You’ll see the Expo Developer Tools open in your browser.

To launch the app on your Android emulator, press:

```bash
a
```

Expo will build your project and open the app on the emulator.

---

**Your Setup is Ready!**

You should now be able to test your entire app — with the frontend connecting to your FastAPI backend via the ngrok tunnel.


## Step 7: Final Deployment of the App

Once your app is stable and bug-free, you can deploy your **FastAPI backend** permanently using [Render](https://render.com).

Render provides free and paid hosting options, making it easy to deploy FastAPI apps with minimal setup.


### Set Up a Render Account

To deploy your backend, the first step is to create a free Render account:

👉 [https://dashboard.render.com/login](https://dashboard.render.com/login)

Render offers free hosting tiers, perfect for testing and lightweight apps. You can upgrade later if needed.

Once signed up, you’ll be ready to connect your project and deploy.


### Add `render.yaml`

In your **backend** folder, create a file called `render.yaml` with the following contents:

```yaml
services:
  - type: web
    name: unit-converter-backend
    env: python
    buildCommand: ""
    startCommand: uvicorn app.main:app --host 0.0.0.0 --port 8000
    plan: free
```

This tells Render how to build and run your FastAPI backend.  
The `plan: free` option keeps it on Render's free tier.

Once this is in place, you’re ready to connect your GitHub repo to Render.

### Deploy to Render

1. Go to the [Render Dashboard](https://dashboard.render.com) and log in if you haven’t already.

2. Click **New** in the top left corner, then select **Web Service**.

3. Choose the **Public Git Repository** tab and paste your repository URL.

4. Set the **Root Directory** to `backend`.

5. For the **Start Command**, enter:

   ```
   uvicorn app.main:app --host 0.0.0.0 --port 8000
   ```

6. Select the **Free** plan.

7. Click **Create Web Service** to start the deployment.


### Test Deployed API

After Render completes the build, you will get a public URL similar to:
`https://unit-converter.onrender.com`

Copy this URL and update the `API_URL` constant in your `ConversionScreen` file in the frontend.

Then, in your frontend folder, run:

```
npx expo start
```

Your app should now connect to the deployed backend and work as expected.


## Step 8: How to Make a Build File

### Prerequisites

* Create a free [Expo account](https://expo.dev) if you don’t have one.
* Install Expo CLI and EAS CLI globally on your machine:

  ```bash
  npm install -g expo-cli
  npm install -g eas-cli
  ```
* Log in to Expo from your terminal:

  ```bash
  eas login
  ```



### Configure App for EAS Build

1. In your frontend project root folder, run:

   ```bash
   eas build:configure
   ```
2. When prompted:

   * For **Automatically create missing build profile?**, select **Yes**
   * Choose **Android** as the platform to configure

This sets up your project with necessary configuration files for EAS build.


### Build `.apk` File for Installation

**Understanding `.apk` vs `.aab`:**

* `.apk` (Android Package) files are ready-to-install Android app files you can sideload directly onto devices or emulators.
* `.aab` (Android App Bundle) files are optimized app bundles submitted to the Google Play Store, which then generate device-specific APKs on the fly. `.aab` is recommended for Play Store but not directly installable.

---

### Update `eas.json` for APK Build

In your frontend project, update the `production` section in `eas.json` to specify building an `.apk` file:

```json
{
  "build": {
    "development": {
      "developmentClient": true,
      "distribution": "internal"
    },
    "preview": {
      "distribution": "internal"
    },
    "production": {
      "android": {
        "buildType": "apk"
      }
    }
  }
}
```

---

### Run the Build

Run this command to start building your APK with the production profile:

```bash
eas build --platform android --profile production
```

* You may be prompted to confirm or change the application ID (package name). Usually, you can accept the default.
* You will be asked whether to generate a new Android Keystore. Say **Yes** to let Expo handle signing keys automatically.

---

### After the Build

Once the build completes (can take several minutes), you will receive a link to download the `.apk` file. You can install this APK on:

* An Android emulator running on your machine
* A physical Android phone (via USB or file transfer)
